In [122]:
import csv
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import itertools as it
from multiprocessing import Queue
from loguru import logger
import time
# this is where any files or directories/path variables go:
csv_filename = './meetings.csv'


## BUILDING A BASIC SCRAPER:
<hr>

In [123]:
# CREATE A CSV PARSER TO GET LINKS FROM CSV FILE:
def csv_parser(csv_reader, header: str):
    _header = next(csv_reader)
    headerIndex = _header.index(header)
    # now create an empty list to append the addresses to
    data_list = []
    # loop through CSV file and append to address_list
    for line in csv_reader:
        all_data = line[headerIndex]
        data_list.append(all_data)
    return data_list

# CREATE A CSV WRITER TO WRITE ROW DATA TO A NEW CSV FILE WITH MEETING DETAILS:
def csv_writer(csv_writer, data):
    # Open CSV file to append data to:
    with open('new_meeting_data.csv', 'w') as f:
        csv_writer = csv.DictWriter(f, field_names=list(data.keys()))


In [124]:
# create func to get list of links:
def get_links(csv_filename):
    with open(csv_filename, 'r') as f:
        csv_reader = csv.reader(f)
        link_list = csv_parser(csv_reader, 'link')
    return link_list 

# create func to get list of links:
def get_addresses(csv_filename):
    with open(csv_filename, 'r') as f:
        csv_reader = csv.reader(f)
        address_list = csv_parser(csv_reader, 'address')
    return address_list

In [136]:
def fetch_soup_data(link):
    # STEP 1: Get the webpage response obj from requests
    page = requests.get(link)
    # STEP 2: Get Soup object for html parsing
    soup = bs(page.text, "lxml")
    logger.info("got soup data for link: {}", link)
    return soup

In [126]:
# CREATE A FUNCTION THAT WILL EXTRACT ADDRESS DATA FROM EACH LINK IN 
# LINK LIST:
def get_address_data(soup):
    
    try:
        address_tag = soup.address
        address = address_tag.contents[1]
        
        meeting_name = soup.find(
            'div', class_='fui-card-body').find(class_='weight-300')
        name = meeting_name.contents[1]
        
        city_tag = meeting_name.find_next('a')
        city = city_tag.contents[0]
        
        state_tag = city_tag.find_next('a')
        state = state_tag.contents[0]
        logger.info("[+] Address data retrieved")
        return {'name': name, 'address': address, 'city': city, 'state': state}
        
    except IndexError as ie:
        logger.error("[-] UnboundError occured {} ", ule)
        try:
            return {'name': name, 'address': address, 'city': city, 'state': 'state'}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundLocalError occured: {} ", ule)
        try:
            return {'name': name, 'address': address, 'city': 'city', 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)
        try:
            return {'name': name, 'address': 'address', 'city': city, 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)
        try:
            return {'name': 'name', 'address': address, 'city': city, 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)

In [137]:
# CREATE A FUNCTION THAT WILL EXTRACT ALL THE TABLE DATA FROM EACH LINK
# IN THE LINK LIST. THE TABLE DATA WILL THEN NEED TO BE PARSED AND 
# CLEANED IF THERE ARE MULTIPLE ITEMS:
def get_table_data(soup):
    try:
        info_table = soup.find('table', class_='table fui-table')
        # obtain all the columns from <th>
        headers = []
        for i in info_table.find_all('th'):
            title = i.text
            headers.append(title.lower())

            # now create a dataframe:
        df = pd.DataFrame(columns=headers)

    
        # Now create the foor loop to fill dataframe
        # a row is under the <tr> tags and data under the <td> tags
        for j in info_table.find_all('tr')[1:]:
            # if info_table.find_all('tr')[1:] == AttributeError.NoneType:
            #     print("No info table found")
            row_data = j.find_all('td')
            row = [i.text for i in row_data]
            length = len(df)
            df.loc[length] = row

        # data['day'].append(df['day'].to_list())
        # data['time'].append(df['time'].to_list())
        # data['info'].append(df['info'].to_list())
        day = df['day'].to_list()
        time = df['time'].to_list()
        info = df['info'].to_list()

        logger.info("[+] Meeting Details Retrieved")
        # now return data
        return {'day': day, 'time': time, 'info': info}
    
    except AttributeError as ae:
        logger.error("info_table.find_all('tr') failed: {}", ae)
        return {'day': 'day', 'time': 'time', 'info': 'info'}

In [138]:
# CREATE A FUNCTION THAT WILL PARSE THE ROW DATA AND STORE IT 
# IN A DICTIONARY. THAT DICTIONARY CAN THEN BE INSERTED INTO 
# A LIST OF DICTIONARIES CALLED ROW_LIST BUT TO DO THIS THE PARSER
# HAS TO JOIN LIST ITEMS INTO ONE LONG STRING SO EACH ROW HAS THE 
# SAME NUMBER OF COLUMNS:
# THIS WAS INSERTED INTO meeting_data_scraper

# VERSION 1
def row_parser1(item0, item1):
    """
    :param item0: This is the address data in a dictionary. Use the following keys to access
    the data -> Keys: 'name' - 'address' - 'city' - 'state' 
    :param item1: This is the meeting details data in a dictionary. Use the following keys to
    access the data -> Keys: 'day' - 'time' - 'info'
    
    create a final dictionary that will be used to store the information in the database as one row. 
    I will need to join the list items to create one string with a | seperating each item so I can 
    split the string when retrieving the data.
    """
    row = {}
    try:
        row['name'] = item0['name']
        row['address'] = item0['address']
        row['city'] = item0['city']
        row['state'] = item0['state']
        logger.info("[+] Row Data Parsed")
    except Exception as e:
        logger.error("[-] Row Data Raised Exception: {}", e)
        print(e)
        row['name'] = 'name'
        row['address'] = 'address'
        row['city'] = 'city'
        row['state'] = 'state'

    # now add item1 to the row data
    row['day'] = ' | '.join(item1['day'])
    row['time'] = ' | '.join(item1['time'])
    row['info'] = ' | '.join(item1['info'])

    # now return the row data dictionary
    
    return row

# VERSION 2: 
def row_parser2(item0, item1):
    """
    :param item0: This is the address data in a dictionary. Use the following keys to access
    the data -> Keys: 'name' - 'address' - 'city' - 'state' 
    :param item1: This is the meeting details data in a dictionary. Use the following keys to
    access the data -> Keys: 'day' - 'time' - 'info'
    
    create a final dictionary that will be used to store the information in the database as one row. 
    I will need to join the list items to create one string with a | seperating each item so I can 
    split the string when retrieving the data.
    """
    try:
        row = {}
        try:
            row['name'] = item0['name']
            row['address'] = item0['address']
            row['city'] = item0['city']
            row['state'] = item0['state']
            # now add item1 to the row data
            row['day'] = ' | '.join(item1['day'])
            row['time'] = ' | '.join(item1['time'])
            row['info'] = ' | '.join(item1['info'])
            # now return the row data dictionary
            return row
        except Exception as e:
            print(f'{e}')
    except Exception as e:
        print(f'{e}')
        for k, v in row.items():
            if v is not None:
                pass
            else:
                v = k
        return row


In [139]:
# THIS IS THE 'MAIN LOGICAL FUNCTION' THIS FUNCTION WILL COMBINE THE 
# get_address_data, get_table_data, and meeting_row_parser FUNCTIONS.
# THAT WAY I CAN EXECUTE ALL OF THE FUNCTIONS IN ONE CALL.
def meeting_data_scraper(link):
    
    # Get Soup Data
    soup = fetch_soup_data(link)
    # Create two dicts with the following keys
    address_dict = get_address_data(soup)
    details_dict = get_table_data(soup)

    d = [address_dict, details_dict]
    row_data = row_parser1(d[0], d[1])

    logger.info("[+] Main Meeting Dart Scraper Worked")
    return row_data


In [140]:
def flatten(list_of_lists):
    "Flatten one level of nesting"
    return it.chain.from_iterable(list_of_lists)

In [131]:
# whole list of links
link_list = get_links(csv_filename)

# shortened list: first 1000
links_1000 = link_list[:1000]

# shortened list: first 500
links_500 = link_list[:500]

links_100 = link_list[:100]


In [141]:
def scrape(link_list):

    soup_data = []
    try:
        count = 0
        for link in link_list:
            soup = fetch_soup_data(link)
            soup_data.append(soup)
            logger.info("[+] {} scraped successfully: Link number {}", link, count)
            count += 1
            link_list.pop()
        return soup_data
    
    except IndexError as e:
        logger.error("{}: List Exhausted. No more links to scrape.", e)

In [142]:
soup_data = scrape(links_100)


2022-06-10 18:16:55.365 | INFO     | __main__:fetch_soup_data:6 - got soup data for link: https://www.aa-meetings.com/aa-meeting/chillicothe-first-capital-group/
2022-06-10 18:16:55.366 | INFO     | __main__:scrape:9 - [+] https://www.aa-meetings.com/aa-meeting/chillicothe-first-capital-group/ scraped successfully: Link number 0
2022-06-10 18:16:55.615 | INFO     | __main__:fetch_soup_data:6 - got soup data for link: https://www.aa-meetings.com/aa-meeting/chillicothe-big-book-study/
2022-06-10 18:16:55.616 | INFO     | __main__:scrape:9 - [+] https://www.aa-meetings.com/aa-meeting/chillicothe-big-book-study/ scraped successfully: Link number 1
2022-06-10 18:16:55.881 | INFO     | __main__:fetch_soup_data:6 - got soup data for link: https://www.aa-meetings.com/aa-meeting/waverly-pike-county-group/
2022-06-10 18:16:55.882 | INFO     | __main__:scrape:9 - [+] https://www.aa-meetings.com/aa-meeting/waverly-pike-county-group/ scraped successfully: Link number 2
2022-06-10 18:16:56.136 | INF

In [143]:
# THIS IS THE 'MAIN LOGICAL FUNCTION' THIS FUNCTION WILL COMBINE THE 
# get_address_data, get_table_data, and meeting_row_parser FUNCTIONS.
# THAT WAY I CAN EXECUTE ALL OF THE FUNCTIONS IN ONE CALL.
def soup_data_scraper(soup_data):
    
    try:
        rows = []
        for soup in soup_data:
            # Create two dicts with the following keys
            address_dict = get_address_data(soup)
            details_dict = get_table_data(soup)

            d = [address_dict, details_dict]
            row_data = row_parser1(d[0], d[1])
            rows.append(row_data)
        logger.info("[+] -----> SUCCESS <------")
        return rows

    except IndexError as e:
        logger.error("{}: List Exhausted. No more Soup Items", e)

In [144]:
rows = soup_data_scraper(soup_data)

2022-06-10 18:17:42.318 | INFO     | __main__:get_address_data:18 - [+] Address data retrieved
2022-06-10 18:17:42.323 | INFO     | __main__:get_table_data:34 - [+] Meeting Details Retrieved
2022-06-10 18:17:42.325 | INFO     | __main__:row_parser1:26 - [+] Row Data Parsed
2022-06-10 18:17:42.327 | INFO     | __main__:get_address_data:18 - [+] Address data retrieved
2022-06-10 18:17:42.333 | INFO     | __main__:get_table_data:34 - [+] Meeting Details Retrieved
2022-06-10 18:17:42.335 | INFO     | __main__:row_parser1:26 - [+] Row Data Parsed
2022-06-10 18:17:42.337 | INFO     | __main__:get_address_data:18 - [+] Address data retrieved
2022-06-10 18:17:42.340 | INFO     | __main__:get_table_data:34 - [+] Meeting Details Retrieved
2022-06-10 18:17:42.340 | INFO     | __main__:row_parser1:26 - [+] Row Data Parsed
2022-06-10 18:17:42.342 | INFO     | __main__:get_address_data:18 - [+] Address data retrieved
2022-06-10 18:17:42.345 | INFO     | __main__:get_table_data:34 - [+] Meeting Detai

In [145]:
def get_rows(link_list):
    try:

        soup_data = scrape(link_list)
        rows = soup_data_scraper(soup_data)
        return rows

    except Exception as e:
        print(f"Exception {e}: Continue running..")

    except requests.exceptions.RequestException as re:
        logger.error("{} Request Exception occured", re)
        time.sleep(10)

In [146]:
row_data = get_rows(link_list)

2022-06-10 18:18:00.832 | INFO     | __main__:fetch_soup_data:6 - got soup data for link: https://www.aa-meetings.com/aa-meeting/chillicothe-first-capital-group/
2022-06-10 18:18:00.833 | INFO     | __main__:scrape:9 - [+] https://www.aa-meetings.com/aa-meeting/chillicothe-first-capital-group/ scraped successfully: Link number 0
2022-06-10 18:18:01.104 | INFO     | __main__:fetch_soup_data:6 - got soup data for link: https://www.aa-meetings.com/aa-meeting/chillicothe-big-book-study/
2022-06-10 18:18:01.105 | INFO     | __main__:scrape:9 - [+] https://www.aa-meetings.com/aa-meeting/chillicothe-big-book-study/ scraped successfully: Link number 1
2022-06-10 18:18:01.378 | INFO     | __main__:fetch_soup_data:6 - got soup data for link: https://www.aa-meetings.com/aa-meeting/waverly-pike-county-group/
2022-06-10 18:18:01.378 | INFO     | __main__:scrape:9 - [+] https://www.aa-meetings.com/aa-meeting/waverly-pike-county-group/ scraped successfully: Link number 2
2022-06-10 18:18:01.645 | INF

KeyboardInterrupt: 

In [ ]:
csv_filename = './meeting_details.csv'
def csv_writer(row_data, csv_filename):
    with open(csv_filename, 'w') as f:   
        writer = csv.writer(f)
        for row in row_data:
            for k,v in row.items():
                writer.writerow(v)

NameError: name 'row_data' is not defined